In this notebook, we implement an autocorrect system. 

In [8]:
import re
import numpy as np
import pandas as pd
from collections import Counter

First, we read the data set and split it to a list of unique words. Also, we make sure all words are lower-case:

In [9]:
file_name = '/content/shakespeare.txt'
with open(file_name) as f:
  data = f.read()

data = data.lower()
words = re.findall('\w+', data)

After the mentioned pre-processes, we now calculate the frequency of each word in the given data set

In [26]:
word_count_dict = Counter(words)

Then, we calculate the probability that each word will appear if randomly selected from the corpus of words

In [27]:
probs = {} # probs is a dictionary: key: word, value: probability

M = sum( word_count_dict.values() ) # Total number of words in the given dataset

for word in word_count_dict.keys():
  probs[word] = word_count_dict[word]/M

# **String Manipulations**

We write several functions to manipulate strings so that we can edit the erroneous strings and return the right spellings of the words.

We implement the following methods:

* `delete_letter`: given a word, it returns all the possible strings that have **one character removed**. 
* `switch_letter`: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* `replace_letter`: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* `insert_letter`: given a word, it returns all the possible strings that have an **additional character inserted**. 

In [28]:
def delete_letter(word):
    '''
    Input:
        word: the string/word for which we will generate all possible words 
                in the vocabulary which have 1 missing character
    Output:
        delete_l: a list of all possible strings obtained by deleting 1 character from word
    '''
    split_l = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    delete_l = [L + R[1:] for (L,R) in split_l if R]

    return  delete_l


def switch_letter(word):
    '''
    Input:
        word: input string
     Output:
        switches: a list of all possible strings with one adjacent charater switched
    ''' 

    split_l = [(word[:i], word[i:]) for i in range(len(word)+1)]
    switch_l = [L + R[1] + R[0] + R[2:] for (L,R) in split_l if len(R) >=2]

    return switch_l



def replace_letter(word):
    '''
    Input:
        word: the input string/word 
    Output:
        replaces: a list of all possible strings where we replaced one letter from the original word. 
    ''' 
    
    letters = 'abcdefghijklmnopqrstuvwxyz'

    split_l = [(word[:i], word[i:]) for i in range(len(word)+1)]
    replace_set = {L + letter + (R[1:] if len(R)>1 else '') for (L,R) in split_l if R for letter in letters }
    replace_set.remove(word)

    replace_l = sorted(list(replace_set)) 
    
    return replace_l




def insert_letter(word):
    '''
    Input:
        word: the input string/word 
    Output:
        inserts: a set of all possible strings with one new letter inserted at every offset
    ''' 
    letters = 'abcdefghijklmnopqrstuvwxyz'
    
    split_l = [(word[:i], word[i:]) for i in range(len(word)+1)]
    insert_l = [L + letter + R for (L,R) in split_l for letter in letters]
    
    return insert_l

In the following, we implement a function to get all the possible edits that are one edit away from the given word. The edits consist of the replace, insert, delete, and the switch operation. 

In [13]:
def edit_one_letter(word, allow_switches = True):
    """
    Input:
        word: the string/word for which we will generate all possible words that are one edit away.
    Output:
        edit_one_set: a set of words with one possible edit. output is a set, not a list!
    """
    
    edit_one_set = set()

    edit_one_set.update(delete_letter(word))
    if allow_switches:
        edit_one_set.update(switch_letter(word))
    edit_one_set.update(replace_letter(word))
    edit_one_set.update(insert_letter(word))

    return set(edit_one_set)



Now, we can generalize this to implement to get two edits on a word:

In [29]:
def edit_two_letters(word, allow_switches = True):
    '''
    Input:
        word: the input string/word 
    Output:
        edit_two_set: a set of strings with all possible two edits
    '''
    
    edit_two_set = set()
    
    edit_one = edit_one_letter(word,allow_switches=allow_switches)
    for w in edit_one:
        if w:
            edit_two = edit_one_letter(w,allow_switches=allow_switches)
            edit_two_set.update(edit_two)
    
    return set(edit_two_set)

In [30]:
def get_corrections(word, probs, vocab, n=2):
    '''
    Input: 
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections we want returned in the dictionary
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''
    
    suggestions = []
    n_best = []
    
    E1 = edit_one_letter(word)
    E2 = edit_two_letters(word)
    
    if word in vocab:
        suggestions.append(word)
    
    if not suggestions:
        for w in E1: 
            if w in vocab:
                suggestions.append(w)
                
    if not suggestions:
        for w in E2:
            if w in vocab:
                suggestions.append(w)
    
                    
    
    sugg_probs = [(w,probs[w]) for w in suggestions]
    sorted_sugg = sorted(sugg_probs, key = lambda x: x[1], reverse = True)
    n_best = sorted_sugg[:n]
    
    
    print("entered word = ", word, "\nsuggestions = ", suggestions)

    return n_best

Now, we can test the performanc of our autocorrect system using misspelled words:

In [43]:
vocab = words
my_word = 'recieve' 
tmp_corrections = get_corrections(my_word, probs, vocab , 2)
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")


entered word =  recieve 
suggestions =  ['receive', 'relieve']
word 0: receive, probability 0.000131
word 1: relieve, probability 0.000019
